In [8]:
import re
import csv
import pandas as pd
import numpy as np

def smoothed_dft(acceleration):
    N = len(acceleration)
    window = np.hanning(N)
    windowed_acceleration = acceleration * window
    dft = np.fft.fft(windowed_acceleration)
    return dft

def calculate_frequency_domain_magnitude(acceleration_array):
    frequency_domain_magnitude = 0
    for acceleration in acceleration_array:
        dft_magnitude = np.abs(smoothed_dft(acceleration))
        frequency_domain_magnitude += pow(dft_magnitude, 2)
    return pow(frequency_domain_magnitude, 1/2)

def calculate_average_phase(acceleration_array):
    sum_imaginary_parts = 0
    sum_real_parts = 0
    for acceleration in acceleration_array:
        dft = smoothed_dft(acceleration)
        sum_real_parts += np.real(dft)
        sum_imaginary_parts += np.imag(dft)
    
    average_phase = np.arctan2(sum_imaginary_parts, sum_real_parts)
    return average_phase

def perform_inverse_dft(magnitude, phase):
    complex_signal = magnitude * np.exp(1j * phase)
    time_domain_signal = np.fft.ifft(complex_signal)
    return time_domain_signal

def generate_time_domain_signal(acceleration_array):
    frequency_domain_magnitude = calculate_frequency_domain_magnitude(acceleration_array)
    average_phase = calculate_average_phase(acceleration_array)
    time_domain_signal = perform_inverse_dft(frequency_domain_magnitude, average_phase)
    return time_domain_signal

# Path to the log file
log_file_path = r'D:\GEM Lab\GEMLabHCIResearch\Unity Logs\TextureData.txt'

# Read the log file
with open(log_file_path, 'r') as file:
    log_data = file.readlines()

# Extract numerical values using regular expression for each line
pattern = r'TEXLOG:\s*([-+]?\d*\.\d+|\d+),\s*([-+]?\d*\.\d+|\d+),\s*([-+]?\d*\.\d+|\d+),\s*([-+]?\d*\.\d+|\d+),\s*([-+]?\d*\.\d+|\d+),\s*([-+]?\d*\.\d+|\d+)'
extracted_values = []

for line in log_data:
    matches = re.search(pattern, line)
    if matches:
        values = matches.groups()
        extracted_values.append(values)

# Calculate DFT321 values based on ADXL-X, ADXL-Y, and ADXL-Z values
dft321_values = []
acceleration_values = []

for row in extracted_values:
    acceleration_array = np.array([[float(row[0])], [float(row[1])], [float(row[2])]])
    acceleration_values.append(acceleration_array)
    dft321_signal = generate_time_domain_signal(acceleration_array)
    dft321_value = np.real(np.mean(dft321_signal))
    dft321_values.append(dft321_value)

# Create a DataFrame for VELO-X, VELO-Y, VELO-Z, and DFT321 values
data = {
    'VELO-X': [row[3] for row in extracted_values],
    'VELO-Y': [row[4] for row in extracted_values],
    'VELO-Z': [row[5] for row in extracted_values],
    'DFT321': dft321_values
}
df = pd.DataFrame(data)

# Calculate the mean DFT321 values for the same VELO-X, VELO-Y, VELO-Z combinations
mean_dft321 = df.groupby(['VELO-X', 'VELO-Y', 'VELO-Z'])['DFT321'].mean().reset_index()

# Merge mean values with the original DataFrame
df = pd.merge(df, mean_dft321, on=['VELO-X', 'VELO-Y', 'VELO-Z'], suffixes=('', '_mean'))
df['DFT321'] = df['DFT321_mean']
df.drop(columns='DFT321_mean', inplace=True)

# Remove duplicate rows based on VELO-X, VELO-Y, VELO-Z, and DFT321 columns
df.drop_duplicates(subset=['VELO-X', 'VELO-Y', 'VELO-Z', 'DFT321'], keep='first', inplace=True)

# Save the DataFrame without duplicate rows to a new CSV file
output_file = r'D:\GEM Lab\GEMLabHCIResearch\Unity based texture CSVs\extracted_values_without_duplicates.csv'
df.to_csv(output_file, index=False)
